### Moon

In [2]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 3.7 MB/s  0:00:02 eta 0:00:01

[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: /Users/hjm/UW_MSDS/.ver3137_venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from difflib import get_close_matches

# 1. 데이터 로드
cosmetics = pd.read_csv('../Datasets/cosmetics.csv')
paula = pd.read_csv('../Datasets/Paula_SUM_LIST.csv')

# 2. 정규화 함수 및 Paula 매핑 구축
def normalize(s):
    s = str(s).lower().replace(" ", "").replace("-", "").replace("_", "")
    return "".join(c for c in s if c.isalnum() or c in ".")

# Paula: ingredient_name -> functions (설명)
paula_desc = dict(zip(paula['ingredient_name'], paula['functions']))
paula_norm_to_orig = {normalize(x): x for x in paula['ingredient_name']}
paula_norm_keys = list(paula_norm_to_orig.keys())

# 3. cosmetics의 모든 유니크 ingredient 수집 후, Paula 매핑 사전 구축 (정확 + fuzzy 0.85)
all_cosmetic_ings = set()
for ing_str in cosmetics['Ingredients'].dropna():
    for x in str(ing_str).split(','):
        x = x.strip()
        if x:
            all_cosmetic_ings.add(x)

ing_to_paula = {}
for ing in all_cosmetic_ings:
    n = normalize(ing)
    if n in paula_norm_to_orig:
        ing_to_paula[ing] = paula_norm_to_orig[n]
    else:
        matches = get_close_matches(n, paula_norm_keys, n=1, cutoff=0.85)
        ing_to_paula[ing] = paula_norm_to_orig[matches[0]] if matches else None

# 4. cosmetics를 ingredient별로 explode (한 행 = 한 성분)
rows = []
for _, row in cosmetics.iterrows():
    ing_str = row.get('Ingredients', '')
    if pd.isna(ing_str) or not str(ing_str).strip():
        continue
    ingredients = [x.strip() for x in str(ing_str).split(',') if x.strip()]
    for ing in ingredients:
        paula_match = ing_to_paula.get(ing)
        description = paula_desc.get(paula_match, '') if paula_match else ''
        rows.append({
            'Label': row['Label'],
            'Brand': row['Brand'],
            'Name': row['Name'],
            'Price': row['Price'],
            'Rank': row['Rank'],
            'Combination': row['Combination'],
            'Dry': row['Dry'],
            'Normal': row['Normal'],
            'Oily': row['Oily'],
            'Sensitive': row['Sensitive'],
            'ingredient': ing,
            'paula_description': description
        })

# 5. 데이터프레임 생성
df_ingredients = pd.DataFrame(rows)
print(f"총 행 수: {len(df_ingredients)}")
print(f"Paula 설명 있음: {(df_ingredients['paula_description'] != '').sum()}개")
print(f"Paula 설명 없음: {(df_ingredients['paula_description'] == '').sum()}개")
df_ingredients

총 행 수: 45812
Paula 설명 있음: 40514개
Paula 설명 없음: 5298개


,Label,Brand,Name,Price,Rank,Combination,Dry,Normal,Oily,Sensitive,ingredient,paula_description
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Algae (Seaweed) Extract,
1,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Mineral Oil,"Clear, odorless oil derived from petroleum tha..."
2,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Petrolatum,Widely considered one of the best and gentlest...
3,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Glycerin,A naturally occurring substance in skin that h...
4,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Isohexadecane,Multitasking ingredient that plays various rol...
...,...,...,...,...,...,...,...,...,...,...,...,...
45807,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,Alpha-isomethyl Ionone,This flavor and fragrance compound is well kno...
45808,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,CI 14700 (Red 4),
45809,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,CI 19140 (Yellow 5),
45810,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,CI 42090 (Blue 1).,


In [4]:
# paula_description에서 어근 추출 → good_for, bad_for 컬럼 추가
import re

# 좋은 효과(이점) 어근 / 나쁜 효과(우려) 어근
GOOD_STEMS = [
    'antioxidant', 'hydrat', 'moistur', 'sooth', 'calm', 'firm', 'smooth', 'brighten',
    'repair', 'protect', 'exfoliat', 'plump', 'soft', 'restore', 'nourish', 'clear',
    'emollient', 'anti-inflamm', 'anti-aging', 'heal', 'strengthen',
    'fade', 'discolor', 'tone', 'balance', 'barrier', 'collagen', 'elastic'
]
BAD_STEMS = [
    'irritat', 'avoid', 'harsh', 'strip', 'drying', 'comedogen', 'allerg',
    'sensitiz', 'sting', 'burn', 'inflam', 'clog', 'pore-clog', 'photosensit'
]

def extract_stems(text, stem_list):
    if pd.isna(text) or not str(text).strip():
        return set()
    text_lower = str(text).lower()
    found = set()
    for stem in stem_list:
        if stem in text_lower:
            found.add(stem)
    return found

def get_good_bad_stems(desc):
    good = extract_stems(desc, GOOD_STEMS)
    bad = extract_stems(desc, BAD_STEMS)
    return good, bad

# 각 행에 good_for, bad_for 추가
good_list = []
bad_list = []
for desc in df_ingredients['paula_description']:
    g, b = get_good_bad_stems(desc)
    good_list.append(','.join(sorted(g)) if g else '')
    bad_list.append(','.join(sorted(b)) if b else '')

df_ingredients['good_for'] = good_list
df_ingredients['bad_for'] = bad_list
df_ingredients

,Label,Brand,Name,Price,Rank,Combination,Dry,Normal,Oily,Sensitive,ingredient,paula_description,good_for,bad_for
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Algae (Seaweed) Extract,,,
1,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Mineral Oil,"Clear, odorless oil derived from petroleum tha...",clear,
2,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Petrolatum,Widely considered one of the best and gentlest...,moistur,
3,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Glycerin,A naturally occurring substance in skin that h...,moistur,
4,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Isohexadecane,Multitasking ingredient that plays various rol...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45807,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,Alpha-isomethyl Ionone,This flavor and fragrance compound is well kno...,,sensitiz
45808,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,CI 14700 (Red 4),,,
45809,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,CI 19140 (Yellow 5),,,
45810,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,20,1.0,0,0,0,0,0,CI 42090 (Blue 1).,,,


In [5]:
# good_for, bad_for 샘플 확인 (paula_description 있는 행만)
sample = df_ingredients[df_ingredients['paula_description'] != ''][['ingredient', 'good_for', 'bad_for', 'paula_description']].head(15)
sample

,ingredient,good_for,bad_for,paula_description
1,Mineral Oil,clear,,"Clear, odorless oil derived from petroleum tha..."
2,Petrolatum,moistur,,Widely considered one of the best and gentlest...
3,Glycerin,moistur,,A naturally occurring substance in skin that h...
4,Isohexadecane,,,Multitasking ingredient that plays various rol...
5,Microcrystalline Wax,smooth,,Used as a thickener and to give products a sem...
6,Lanolin Alcohol,emollient,,Emollient derived from lanolin.
7,Citrus Aurantifolia (Lime) Extract,,,Functions: Skin Conditioning. We have not yet ...
8,Sesamum Indicum (Sesame) Seed Oil,"antioxidant,calm,moistur",,Non-fragrant plant oil that is a good source o...
10,Sesamum Indicum (Sesame) Seed Powder,,,Functions: Skin Conditioning. We have not yet ...
11,Medicago Sativa (Alfalfa) Seed Powder,,,Functions: Skin Conditioning. We have not yet ...


In [6]:
# good_for, bad_for 어근을 각 행으로 explode + 5개 스킨타입 컬럼(1/-1/0) 추가
# 근거: 피부과학 일반 지식 (hydrat→건성에 좋음, exfoliat→지성에 좋고 민감성에 나쁨 등)

# 어근별 스킨타입 매핑 (Combination, Dry, Normal, Oily, Sensitive)
# 1=좋음, -1=나쁨, 0=영향없음
SKIN_TYPE_MAP = {
    # good_for 어근
    'antioxidant': (1, 1, 1, 1, 1),
    'hydrat': (1, 1, 1, 0, 1),
    'moistur': (1, 1, 1, 0, 1),
    'sooth': (1, 1, 1, 0, 1),
    'calm': (1, 1, 1, 0, 1),
    'firm': (1, 1, 1, 0, 0),
    'smooth': (1, 1, 1, 1, 1),
    'brighten': (1, 1, 1, 1, 1),
    'repair': (1, 1, 1, 1, 1),
    'protect': (1, 1, 1, 1, 1),
    'exfoliat': (1, -1, 1, 1, -1),
    'plump': (1, 1, 1, 0, 1),
    'soft': (1, 1, 1, 0, 1),
    'restore': (1, 1, 1, 1, 1),
    'nourish': (1, 1, 1, 0, 1),
    'clear': (1, 0, 1, 1, 0),
    'emollient': (1, 1, 1, 0, 1),
    'anti-inflamm': (1, 1, 1, 0, 1),
    'anti-aging': (1, 1, 1, 0, 0),
    'heal': (1, 1, 1, 1, 1),
    'strengthen': (1, 1, 1, 1, 1),
    'fade': (1, 1, 1, 1, 1),
    'discolor': (1, 1, 1, 1, 1),
    'tone': (1, 1, 1, 1, 1),
    'balance': (1, 1, 1, 1, 1),
    'barrier': (1, 1, 1, 1, 1),
    'collagen': (1, 1, 1, 0, 0),
    'elastic': (1, 1, 1, 0, 0),
    # bad_for 어근
    'irritat': (-1, -1, -1, 0, -1),
    'avoid': (-1, -1, -1, -1, -1),
    'harsh': (-1, -1, -1, 0, -1),
    'strip': (-1, -1, -1, 0, -1),
    'drying': (-1, -1, 0, 1, -1),
    'comedogen': (0, 0, -1, -1, -1),
    'allerg': (-1, -1, -1, -1, -1),
    'sensitiz': (0, 0, 0, 0, -1),
    'sting': (0, 0, 0, 0, -1),
    'burn': (0, 0, 0, 0, -1),
    'inflam': (-1, -1, -1, 0, -1),
    'clog': (0, 0, -1, -1, -1),
    'pore-clog': (0, 0, -1, -1, -1),
    'photosensit': (-1, -1, -1, -1, -1),
}

SKIN_COLS = ['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']

def get_skin_values(stem):
    """어근별 스킨타입 효과 반환 (1=좋음, -1=나쁨, 0=영향없음)"""
    vals = SKIN_TYPE_MAP.get(stem)
    return list(vals) if vals else [0, 0, 0, 0, 0]

# explode: good_for, bad_for가 비어있지 않은 행만, 각 어근별로 새 행 생성
rows_expanded = []
for idx, row in df_ingredients.iterrows():
    good_stems = [s.strip() for s in str(row['good_for']).split(',') if s.strip()]
    bad_stems = [s.strip() for s in str(row['bad_for']).split(',') if s.strip()]
    
    if not good_stems and not bad_stems:
        # 비어있으면 5개 스킨타입 효과 열도 공란(NaN)
        base = row.to_dict()
        base['effect_stem'] = ''
        base['effect_type'] = ''
        for c in SKIN_COLS:
            base[f'{c}_effect'] = None  # 공란
        rows_expanded.append(base)
        continue
    
    for stem in good_stems:
        base = row.to_dict()
        base['effect_stem'] = stem
        base['effect_type'] = 'good'
        vals = get_skin_values(stem)
        for i, col in enumerate(SKIN_COLS):
            base[f'{col}_effect'] = vals[i]
        rows_expanded.append(base)
    
    for stem in bad_stems:
        base = row.to_dict()
        base['effect_stem'] = stem
        base['effect_type'] = 'bad'
        vals = get_skin_values(stem)
        for i, col in enumerate(SKIN_COLS):
            base[f'{col}_effect'] = vals[i]
        rows_expanded.append(base)

df_effects = pd.DataFrame(rows_expanded)
print(f"기존 df_ingredients: {len(df_ingredients)}행 × {len(df_ingredients.columns)}열")
print(f"변환 후 df_effects: {len(df_effects)}행 × {len(df_effects.columns)}열")
print(f"추가된 스킨타입 효과 열: {[c for c in df_effects.columns if c.endswith('_effect')]}")
df_effects.head(20)

기존 df_ingredients: 45812행 × 14열
변환 후 df_effects: 60836행 × 21열
추가된 스킨타입 효과 열: ['Combination_effect', 'Dry_effect', 'Normal_effect', 'Oily_effect', 'Sensitive_effect']


,Label,Brand,Name,Price,Rank,Combination,Dry,Normal,Oily,Sensitive,...,paula_description,good_for,bad_for,effect_stem,effect_type,Combination_effect,Dry_effect,Normal_effect,Oily_effect,Sensitive_effect
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,,,,,,NaN,NaN,NaN,NaN,NaN
1,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,"Clear, odorless oil derived from petroleum tha...",clear,,clear,good,1.0,0.0,1.0,1.0,0.0
2,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,Widely considered one of the best and gentlest...,moistur,,moistur,good,1.0,1.0,1.0,0.0,1.0
3,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,A naturally occurring substance in skin that h...,moistur,,moistur,good,1.0,1.0,1.0,0.0,1.0
4,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,Multitasking ingredient that plays various rol...,,,,,NaN,NaN,NaN,NaN,NaN
5,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,Used as a thickener and to give products a sem...,smooth,,smooth,good,1.0,1.0,1.0,1.0,1.0
6,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,Emollient derived from lanolin.,emollient,,emollient,good,1.0,1.0,1.0,0.0,1.0
7,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,Functions: Skin Conditioning. We have not yet ...,,,,,NaN,NaN,NaN,NaN,NaN
8,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,Non-fragrant plant oil that is a good source o...,"antioxidant,calm,moistur",,antioxidant,good,1.0,1.0,1.0,1.0,1.0
9,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,...,Non-fragrant plant oil that is a good source o...,"antioxidant,calm,moistur",,calm,good,1.0,1.0,1.0,0.0,1.0


In [7]:
# 컬럼 요약
print("=== df_effects 컬럼 구성 ===")
print(f"총 컬럼 수: {len(df_effects.columns)}개")
print()
print("1. 기존 df_ingredients 컬럼 (12개):")
orig_cols = ['Label','Brand','Name','Price','Rank','Combination','Dry','Normal','Oily','Sensitive','ingredient','paula_description','good_for','bad_for']
print(f"   {orig_cols}")
print()
print("2. 추가된 effect 관련 컬럼 (3개):")
print("   effect_stem, effect_type")
print()
print("3. 스킨타입 효과 열 (5개): 1=좋음, -1=나쁨, 0=영향없음")
print(f"   {[c for c in df_effects.columns if c.endswith('_effect')]}")
print()
print("총: 14(기존+good_for,bad_for) + 2(effect_stem,effect_type) + 5(스킨타입_effect) = 21개")
df_effects[['ingredient','effect_stem','effect_type','Combination_effect','Dry_effect','Normal_effect','Oily_effect','Sensitive_effect']].head(25)

=== df_effects 컬럼 구성 ===
총 컬럼 수: 21개

1. 기존 df_ingredients 컬럼 (12개):
   ['Label', 'Brand', 'Name', 'Price', 'Rank', 'Combination', 'Dry', 'Normal', 'Oily', 'Sensitive', 'ingredient', 'paula_description', 'good_for', 'bad_for']

2. 추가된 effect 관련 컬럼 (3개):
   effect_stem, effect_type

3. 스킨타입 효과 열 (5개): 1=좋음, -1=나쁨, 0=영향없음
   ['Combination_effect', 'Dry_effect', 'Normal_effect', 'Oily_effect', 'Sensitive_effect']

총: 14(기존+good_for,bad_for) + 2(effect_stem,effect_type) + 5(스킨타입_effect) = 21개


,ingredient,effect_stem,effect_type,Combination_effect,Dry_effect,Normal_effect,Oily_effect,Sensitive_effect
0,Algae (Seaweed) Extract,,,NaN,NaN,NaN,NaN,NaN
1,Mineral Oil,clear,good,1.0,0.0,1.0,1.0,0.0
2,Petrolatum,moistur,good,1.0,1.0,1.0,0.0,1.0
3,Glycerin,moistur,good,1.0,1.0,1.0,0.0,1.0
4,Isohexadecane,,,NaN,NaN,NaN,NaN,NaN
5,Microcrystalline Wax,smooth,good,1.0,1.0,1.0,1.0,1.0
6,Lanolin Alcohol,emollient,good,1.0,1.0,1.0,0.0,1.0
7,Citrus Aurantifolia (Lime) Extract,,,NaN,NaN,NaN,NaN,NaN
8,Sesamum Indicum (Sesame) Seed Oil,antioxidant,good,1.0,1.0,1.0,1.0,1.0
9,Sesamum Indicum (Sesame) Seed Oil,calm,good,1.0,1.0,1.0,0.0,1.0


In [8]:
# df_effects 데이터프레임을 CSV 파일로 저장
df_effects.to_csv('../Datasets/ingredient_effects.csv', index=False, encoding='utf-8')
print(f"저장 완료: Datasets/ingredient_effects.csv")
print(f"  - 행: {len(df_effects):,}개")
print(f"  - 열: {len(df_effects.columns)}개")

저장 완료: Datasets/ingredient_effects.csv
  - 행: 60,836개
  - 열: 21개


In [9]:
# 임의로 10개 샘플: 제품+성분+스킨타입 효과+성분 설명
sample_cols = ['Brand', 'Name', 'ingredient', 'effect_stem', 'Combination_effect', 'Dry_effect', 'Normal_effect', 'Oily_effect', 'Sensitive_effect', 'paula_description']
df_sample = df_effects[sample_cols].sample(n=min(10, len(df_effects)), random_state=42)
df_sample

,Brand,Name,ingredient,effect_stem,Combination_effect,Dry_effect,Normal_effect,Oily_effect,Sensitive_effect,paula_description
34979,ORIGINS,Original Skin™ Retexturizing Mask with Rose Clay,Xanthan Gum,,NaN,NaN,NaN,NaN,NaN,This naturally derived ingredient is produced ...
39908,DR. JART+,Dermask Ultra Jet Porecting Solution Bubbling ...,Calcium Clubonate,sooth,1.0,1.0,1.0,0.0,1.0,Calcium is an essential mineral for the body. ...
35937,SEPHORA COLLECTION,Face Mask - Avocado,Phenoxyethanol,protect,1.0,1.0,1.0,1.0,1.0,Safe preservative known to be well-tolerated b...
57137,ERBORIAN,CC Red Correct Automatic Perfector Broad Spect...,Silica Dimethyl Silylate,,NaN,NaN,NaN,NaN,NaN,Used as a slip and suspending agent.
36564,DR. JART+,Shake & Shot™ Rubber Masks,Niacinamide,hydrat,1.0,1.0,1.0,0.0,1.0,"Helps visibly minimize pore size, improve dull..."
52465,KATE SOMERVILLE,Age Arrest Eye Cream,Propanediol,moistur,1.0,1.0,1.0,0.0,1.0,Improves ingredient absorption while adding mo...
6935,ORIGINS,A Perfect World™ SPF 40 Age-Defense Moisturize...,Algae Extract,,NaN,NaN,NaN,NaN,NaN,"The extract of very simple, chlorophyll-contai..."
16295,NEOGEN DERMALOGY,Green Tea Real Fresh Foam Cleanser,Methylglucamine,,NaN,NaN,NaN,NaN,NaN,Functions: Hair Conditioning. We have not yet ...
7499,ORIGINS,Dr. Andrew Weil For Origins™ Mega-Bright SPF 3...,Punica Granatum (Pomegranate) Fruit Juice,,NaN,NaN,NaN,NaN,NaN,
7277,MURAD,Oil-Control Mattifier SPF 15 PA++,Betaine,moistur,1.0,1.0,1.0,0.0,1.0,"Helps skin adapt to moisture losses and gains,..."


In [10]:
# 데이터프레임 head 확인
df_ingredients.head(20)

,Label,Brand,Name,Price,Rank,Combination,Dry,Normal,Oily,Sensitive,ingredient,paula_description,good_for,bad_for
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Algae (Seaweed) Extract,,,
1,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Mineral Oil,"Clear, odorless oil derived from petroleum tha...",clear,
2,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Petrolatum,Widely considered one of the best and gentlest...,moistur,
3,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Glycerin,A naturally occurring substance in skin that h...,moistur,
4,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Isohexadecane,Multitasking ingredient that plays various rol...,,
5,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Microcrystalline Wax,Used as a thickener and to give products a sem...,smooth,
6,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Lanolin Alcohol,Emollient derived from lanolin.,emollient,
7,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Citrus Aurantifolia (Lime) Extract,Functions: Skin Conditioning. We have not yet ...,,
8,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Sesamum Indicum (Sesame) Seed Oil,Non-fragrant plant oil that is a good source o...,"antioxidant,calm,moistur",
9,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,1,1,1,1,Eucalyptus Globulus (Eucalyptus) Leaf Oil,,,
